In [1]:
from datasets import load_dataset

train_ds, val_ds, test_ds = load_dataset('imdb', split=['train', 'test[:20%]','test[20%:]'])

Reusing dataset imdb (C:\Users\AR88750\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [3]:
val_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [4]:
test_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-e5cca6ad08bef035.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-5b18ee2e8674114e.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a\cache-31b4515ba23880b3.arrow


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments("trainer", evaluation_strategy="epoch")

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 25000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9375


Epoch,Training Loss,Validation Loss
1,0.315500,0.221880
2,0.202000,0.345280
3,0.101400,0.364472


Saving model checkpoint to trainer\checkpoint-500
Configuration saved in trainer\checkpoint-500\config.json
Model weights saved in trainer\checkpoint-500\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-1000
Configuration saved in trainer\checkpoint-1000\config.json
Model weights saved in trainer\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-1500
Configuration saved in trainer\checkpoint-1500\config.json
Model weights saved in trainer\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-2000
Configuration saved in trainer\checkpoint-2000\config.json
Model weights saved in trainer\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-2500
Configuration saved in trainer\checkpoint-2500\config.json
Model weights saved in trainer\checkpoint-2500\pytorch_model.bin
Saving model checkpoint to trainer\checkpoint-3000
Configuration saved in trainer\checkpoint-3000\config.json
Model weights save

TrainOutput(global_step=9375, training_loss=0.2375319580078125, metrics={'train_runtime': 4312.2908, 'train_samples_per_second': 17.392, 'train_steps_per_second': 2.174, 'total_flos': 1.9733329152e+16, 'train_loss': 0.2375319580078125, 'epoch': 3.0})

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 8


{'eval_loss': 0.3688938617706299,
 'eval_accuracy': 0.9252,
 'eval_runtime': 356.9865,
 'eval_samples_per_second': 56.025,
 'eval_steps_per_second': 7.003}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
)
trainer.evaluate()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

training_args = TrainingArguments("full_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)
trainer.train()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from cache at /

Epoch,Training Loss,Validation Loss
1,0.332700,0.295111
2,0.225900,0.332293
3,0.102800,0.366372


Saving model checkpoint to full_trainer/checkpoint-500
Configuration saved in full_trainer/checkpoint-500/config.json
Model weights saved in full_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to full_trainer/checkpoint-1000
Configuration saved in full_trainer/checkpoint-1000/config.json
Model weights saved in full_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to full_trainer/checkpoint-1500
Configuration saved in full_trainer/checkpoint-1500/config.json
Model weights saved in full_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to full_trainer/checkpoint-2000
Configuration saved in full_trainer/checkpoint-2000/config.json
Model weights saved in full_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to full_trainer/checkpoint-2500
Configuration saved in full_trainer/checkpoint-2500/config.json
Model weights saved in full_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to full_trainer/checkpoint-30

TrainOutput(global_step=9375, training_loss=0.2477896944173177, metrics={'train_runtime': 19390.9336, 'train_samples_per_second': 3.868, 'train_steps_per_second': 0.483, 'total_flos': 1.9733329152e+16, 'train_loss': 0.2477896944173177, 'epoch': 3.0})

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8


{'eval_accuracy': 0.9258,
 'eval_loss': 0.36637213826179504,
 'eval_runtime': 1705.8043,
 'eval_samples_per_second': 14.656,
 'eval_steps_per_second': 1.832}

In [ ]:
model.save_pretrained("my_imdb_model")

Configuration saved in my_imdb_model/config.json
Model weights saved in my_imdb_model/pytorch_model.bin
